In [1]:
#default_exp sklearn

In [2]:
#TODO: add "toggle_cond" to HPToggle

In [3]:
#export
from hypster.oo_hp import *
from hypster.hypster_prepare import *

import fastai2
from fastai2.tabular.all import *
from fastai2.metrics import *

from sklearn.model_selection import train_test_split

from copy import deepcopy

import optuna

In [4]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer

In [5]:
from sklearn.impute import SimpleImputer

In [6]:
from sklearn.preprocessing import OneHotEncoder

In [7]:
from sklearn.ensemble import RandomForestClassifier

In [8]:
from category_encoders import BinaryEncoder

In [9]:
#export
SEED = 42

# Read Data

In [10]:
#export
path = untar_data(URLs.ADULT_SAMPLE)
path.ls()

(#3) [Path('C:/Users/user/.fastai/data/adult_sample/adult.csv'),Path('C:/Users/user/.fastai/data/adult_sample/export.pkl'),Path('C:/Users/user/.fastai/data/adult_sample/models')]

In [11]:
#export
df = pd.read_csv(path/'adult.csv')
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,49,Private,101320,Assoc-acdm,12.0,Married-civ-spouse,NaN,Wife,White,Female,0,1902,40,United-States,>=50k
1,44,Private,236746,Masters,14.0,Divorced,Exec-managerial,Not-in-family,White,Male,10520,0,45,United-States,>=50k
2,38,Private,96185,HS-grad,NaN,Divorced,NaN,Unmarried,Black,Female,0,0,32,United-States,<50k
3,38,Self-emp-inc,112847,Prof-school,15.0,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,0,0,40,United-States,>=50k
4,42,Self-emp-not-inc,82297,7th-8th,NaN,Married-civ-spouse,Other-service,Wife,Black,Female,0,0,50,United-States,<50k


In [12]:
#df = df.sample(frac=0.1)

In [13]:
#export
cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race']
cont_names = ['age', 'fnlwgt', 'education-num']
dep_var = "salary"

In [14]:
y = np.where(df[dep_var] == ">=50k", 1, 0)
df.drop(dep_var, axis=1, inplace=True)

In [15]:
#export
train_df, test_df, y_train, y_test = train_test_split(df, y, test_size=0.3, 
                                                     random_state=SEED, 
                                                     stratify=y)

# Preprocessing

In [16]:
train_df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
24092,43,Self-emp-not-inc,118261,Masters,14.0,Divorced,Other-service,Unmarried,White,Female,0,0,30,United-States
8881,42,Private,187294,Some-college,10.0,Divorced,Craft-repair,Unmarried,White,Male,0,0,40,United-States
15643,20,Private,436253,Some-college,10.0,Never-married,Other-service,Own-child,White,Male,0,0,35,United-States
9656,63,Private,264968,Bachelors,13.0,Never-married,Adm-clerical,Not-in-family,White,Male,0,0,40,United-States
7266,46,Private,205100,Assoc-voc,11.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,48,United-States


In [17]:
RandomForestClassifier = prepare(RandomForestClassifier)
SimpleImputer = prepare(SimpleImputer)
Pipeline = prepare(Pipeline)
ColumnTransformer = prepare(ColumnTransformer)
FeatureUnion = prepare(FeatureUnion)

In [18]:
imputer = SimpleImputer(strategy=HpCategorical(["mean", "median"]))

In [19]:
cat_enc = BinaryEncoder()

In [32]:
#test

In [20]:
rf = RandomForestClassifier(n_estimators=HpInt(10, 20, 10),
                            criterion=HpCategorical(['gini', 'entropy']),
                            max_depth=HpToggle(HpInt(5, 1000, 5)),
                            max_leaf_nodes=HpToggle(HpInt(2, 1000)),
                            max_features=HpToggle(HpCategorical(['auto', 'sqrt', 'log2', None])),
                            #min_impurity_decrease=0.0,
                            bootstrap=HpBool(),
                            oob_score=False,
                            n_jobs=-1,
                            random_state=SEED,
                            verbose=0,
                            warm_start=False,
                            class_weight=None,
                            #ccp_alpha=0.0,
                            #max_samples=None
                            )

In [21]:
cont_imp = ColumnTransformer([("imp", imputer, cont_names)])

In [22]:
cat_enc = ColumnTransformer([("cat_enc", cat_enc, cat_names)])

In [23]:
fe = FeatureUnion([("cat", cat_enc), ("cont", cont_imp)])

In [24]:
pipe = Pipeline([("fe", fe), ("model", rf)])

In [25]:
#export
import datetime
def run_learner(fit_method, get_metric, n_trials=5): #learner
    class Objective():
        def __init__(self, fit_method, get_metric): #learner
            #self.learner   = learner
            self.fit_method = fit_method
            self.get_metric = get_metric
            
        def __call__(self, trial): 
            #learner = self.learner.sample(trial)
            self.fit_method.sample(trial)
            res = self.get_metric.sample(trial)
            #print(self.fit_method.base_call)
            #print(self.get_metric.base_call.base_call)
            print(res)
            return res

    objective = Objective(fit_method, get_metric) #learner
    optuna.logging.set_verbosity(0)
    pruner = optuna.pruners.NopPruner()
    now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    study = optuna.create_study(direction="maximize", study_name = now, pruner=pruner)
    study.optimize(objective, n_trials=n_trials, n_jobs=1, timeout=600)
    return study

In [26]:
#export
study = run_learner(#learner    = learner,
                    fit_method = pipe.fit(train_df, y_train),
                    get_metric = pipe.score(test_df, y_test),
                    n_trials   = 10
                   )

0.8111372709591566
0.8277203398505476
0.8305865492885659
0.8258777766403931
0.8072474152932746
0.8339645818405159
0.7978298699969291
0.8111372709591566
0.7692701402395332
0.8300747261746341


In [27]:
#export
print("Number of finished trials: {}".format(len(study.trials)))

Number of finished trials: 10


In [28]:
print("Best trial:")
trial = study.best_trial
print("  Value: {}".format(trial.value))

Best trial:
  Value: 0.8339645818405159


In [29]:
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

  Params: 
    strategy: median
    n_estimators: 10
    criterion: entropy
    toggle_max_depth: False
    toggle_max_leaf_nodes: True
    max_leaf_nodes: 949
    toggle_max_features: False
    bootstrap: False


In [30]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,params_bootstrap,params_criterion,params_max_depth,params_max_features,params_max_leaf_nodes,params_n_estimators,params_strategy,params_toggle_max_depth,params_toggle_max_features,params_toggle_max_leaf_nodes,state
0,0,0.811137,2020-06-19 18:24:24.670801,2020-06-19 18:24:25.228800,True,gini,NaN,auto,NaN,10,mean,False,True,False,COMPLETE
1,1,0.827720,2020-06-19 18:24:25.229801,2020-06-19 18:24:26.071951,True,entropy,NaN,None,934.0,20,median,False,True,True,COMPLETE
2,2,0.830587,2020-06-19 18:24:26.072952,2020-06-19 18:24:26.802539,True,entropy,155.0,None,320.0,20,median,True,True,True,COMPLETE
3,3,0.825878,2020-06-19 18:24:26.802539,2020-06-19 18:24:27.280543,True,entropy,25.0,NaN,53.0,10,median,True,False,True,COMPLETE
4,4,0.807247,2020-06-19 18:24:27.281511,2020-06-19 18:24:27.776511,True,entropy,NaN,NaN,NaN,10,mean,False,False,False,COMPLETE
5,5,0.833965,2020-06-19 18:24:27.777512,2020-06-19 18:24:28.268512,False,entropy,NaN,NaN,949.0,10,median,False,False,True,COMPLETE
6,6,0.797830,2020-06-19 18:24:28.270513,2020-06-19 18:24:28.878347,False,entropy,440.0,NaN,NaN,20,mean,True,False,False,COMPLETE
7,7,0.811137,2020-06-19 18:24:28.879346,2020-06-19 18:24:29.354965,True,gini,NaN,sqrt,NaN,10,mean,False,True,False,COMPLETE
8,8,0.769270,2020-06-19 18:24:29.356962,2020-06-19 18:24:30.130446,False,entropy,965.0,None,NaN,10,median,True,True,False,COMPLETE
9,9,0.830075,2020-06-19 18:24:30.131447,2020-06-19 18:24:30.598033,False,entropy,860.0,NaN,523.0,10,mean,True,False,True,COMPLETE
